# HAS Tools
### Forecast 15 - Danielle Tadych

In [2]:

import numpy as np
import pandas as pd

def getflowyr(data_set, year):
    'This function can create a pandas data frame filtered by year.'
    q = pd.DataFrame(data_set[data_set['year'] == year])
    print(q)
    return q


def getflowyrmo(data_set, year, month):
    'Can create a pandas dataframe filtered by year and month.'
    q = pd.DataFrame(data_set[(data_set['year'] == year)
                              & (data_set['month'] == month)])
    print(q)
    return q


def getflowymw(data_set, year, month, week):
    'Can create a pandas data frame filtered by year, month, and week'
    q = pd.DataFrame(data_set[(data_set['year'] == year)
                              & (data_set['month'] == month)
                              & (data_set['weeknumber'] == week)])
    print(q)
    return q

url = 'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=' \
      'rdb&site_no=09506000&referred_module=sw&period=&begin_date' \
      '=1989-01-01&end_date=2020-12-5'

# Read the data into a pandas dataframe

data = pd.read_table(url, sep='\t', skiprows=30,
                     names=['agency_cd', 'site_no', 'datetime', 'flow',
                            'code'], parse_dates=['datetime'], 
                     )

# Expand the dates to year month day
data['year'] = pd.DatetimeIndex(data['datetime']).year
data['month'] = pd.DatetimeIndex(data['datetime']).month
data['day'] = pd.DatetimeIndex(data['datetime']).dayofweek
data['dayofweek'] = pd.DatetimeIndex(data['datetime']).dayofweek

# Aggregate flow values to week and subset full dataset
flow_weekly = data.resample("W", on='datetime').mean()
observedweeklyflow = flow_weekly

# Actual Forecast

# Only going to use 2020 October daily flows to predict
# Since it's been increasing slightly, I'm using the october mean
# for week 1 forecast and mean plus the standard deviation for week 2
november = getflowyrmo(data, 2020, 11)
december = getflowyrmo(data, 2020, 12)
november = november.append(december)
#print(november)

week1_forecast = np.mean(november['flow'].tail(2))
week2_forecast = week1_forecast + np.std(november['flow'])

# Seasonal Forecast

# Just going to use the last months of 2019 since it also
# is a record dry year
semestermonths = [8, 9, 10, 11, 12]
semester2009 = pd.DataFrame()

for i in semestermonths:
    semesterflow = getflowyrmo(observedweeklyflow, 2009, i)
    semester2009 = semester2009.append(semesterflow)

      agency_cd  site_no   datetime   flow code  year  month  day  dayofweek
11627      USGS  9506000 2020-11-01  124.0    P  2020     11    6          6
11628      USGS  9506000 2020-11-02  123.0    P  2020     11    0          0
11629      USGS  9506000 2020-11-03  118.0    P  2020     11    1          1
11630      USGS  9506000 2020-11-04  121.0    P  2020     11    2          2
11631      USGS  9506000 2020-11-05  126.0    P  2020     11    3          3
11632      USGS  9506000 2020-11-06  127.0    P  2020     11    4          4
11633      USGS  9506000 2020-11-07  127.0    P  2020     11    5          5
11634      USGS  9506000 2020-11-08  134.0    P  2020     11    6          6
11635      USGS  9506000 2020-11-09  149.0    P  2020     11    0          0
11636      USGS  9506000 2020-11-10  151.0    P  2020     11    1          1
11637      USGS  9506000 2020-11-11  154.0    P  2020     11    2          2
11638      USGS  9506000 2020-11-12  154.0    P  2020     11    3          3

In [3]:
print()
print("Actual Forecast:")
print("Week 1: ", np.round(week1_forecast, decimals=2), "cfs")
print("Week 2: ", np.round(week2_forecast, decimals=2), "cfs")
print()
print("Seasonal Forecast:")
print(semester2009)


Actual Forecast:
Week 1:  160.5 cfs
Week 2:  173.11 cfs

Seasonal Forecast:
              site_no        flow    year  month  day  dayofweek
datetime                                                        
2009-08-09  9506000.0   37.700000  2009.0    8.0  3.0        3.0
2009-08-16  9506000.0   47.842857  2009.0    8.0  3.0        3.0
2009-08-23  9506000.0   46.942857  2009.0    8.0  3.0        3.0
2009-08-30  9506000.0   75.442857  2009.0    8.0  3.0        3.0
2009-09-13  9506000.0  182.285714  2009.0    9.0  3.0        3.0
2009-09-20  9506000.0  119.342857  2009.0    9.0  3.0        3.0
2009-09-27  9506000.0  102.271429  2009.0    9.0  3.0        3.0
2009-10-11  9506000.0   95.542857  2009.0   10.0  3.0        3.0
2009-10-18  9506000.0  102.485714  2009.0   10.0  3.0        3.0
2009-10-25  9506000.0  103.042857  2009.0   10.0  3.0        3.0
2009-11-08  9506000.0  134.142857  2009.0   11.0  3.0        3.0
2009-11-15  9506000.0  138.714286  2009.0   11.0  3.0        3.0
2009-11-22  9